# Ćwiczenie: Budowa core DataBota

**Czas trwania:** 90 minut

**Cel ćwiczenia:** W tym ćwiczeniu stworzymy szkielet DataBota zintegrowany z Mosaic Vector Search, który będzie w stanie przetwarzać zapytania użytkownika, pobierać odpowiednie dane i generować odpowiedzi przy użyciu modelu LLM.

## Integracja z Mosaic Vector Search i embeddingi DBRX (Databricks)

In [0]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/41.9 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.0 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/437.7 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/360.3 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.3/360.3 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.3 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 3.3/3.3 MB 144.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/603.9 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.9/603.9 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [0]:
from langchain.document_loaders import PyPDFLoader

# Załaduj dokument PDF
loader = PyPDFLoader("example.pdf")
documents = loader.load()
print(f"Załadowano {len(documents)} stron")

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File /local_disk0/.ephemeral_nfs/envs/pythonEnv-b6484e2e-695e-4af7-9646-c66ea3f45576/lib/python3.12/site-packages/langchain/_api/module_import.py:69, in create_importer.<locals>.import_by_name(name)
     68 try:
---> 69     module = importlib.import_module(new_module)
     70 except ModuleNotFoundError as e:

File /usr/lib/python3.12/importlib/__init__.py:90, in import_module(name, package)
     89         level += 1
---> 90 return _bootstrap._gcd_import(name[level:], package, level)

File <frozen importlib._bootstrap>:1387, in _gcd_import(name, package, level)

File <frozen importlib._bootstrap>:1360, in _find_and_load(name, import_)

File <frozen importlib._bootstrap>:1310, in _find_and_load_unlocked(name, import_)

File <frozen importlib._bootstrap>:488, in _call_with_frames_removed(f, *args, **kwds)

File <frozen importlib._bootstra

In [0]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Podział na fragmenty
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)
print(f"Podzielono na {len(docs)} fragmentów")

In [0]:
from mlflow.deployments import get_deploy_client

texts = ["example text 1", "example text 2"]

client = get_deploy_client("databricks")

# Call the embedding endpoint
response = client.predict(endpoint="databricks-gte-large-en", inputs={"input": texts})

# Access embeddings
embeddings = response['data']
print(embeddings)

[{'index': 0, 'object': 'embedding', 'embedding': [-0.39111328125, 0.5419921875, -0.4560546875, -0.451416015625, -0.36474609375, 0.41015625, 0.01099395751953125, -0.287841796875, -0.83203125, 0.14501953125, -0.69970703125, -0.283203125, 0.08331298828125, 0.11627197265625, -0.344970703125, -1.1123046875, -0.3623046875, -0.84228515625, 0.7958984375, -1.28515625, -0.1739501953125, 0.164794921875, 0.261962890625, -2.216796875, 0.454833984375, -0.34326171875, -0.7197265625, -0.032867431640625, 0.5029296875, 0.6875, -0.2493896484375, 0.1510009765625, -0.227294921875, 0.11651611328125, -0.245361328125, 0.86767578125, 0.001186370849609375, 0.11749267578125, -0.485107421875, -1.029296875, -0.3857421875, -0.259033203125, 0.40478515625, -0.84033203125, -0.284912109375, -0.0914306640625, 0.333740234375, -0.2095947265625, 0.13232421875, 0.7509765625, -1.267578125, 1, 0.1329345703125, -0.267333984375, 0.287109375, 0.418701171875, -0.1199951171875, -0.215576171875, -0.146728515625, -0.324951171875, -

In [0]:
from mlflow.deployments import get_deploy_client

# Użyj endpointu z modelem embeddingującym (np. DBRX)
client = get_deploy_client("databricks")

# Konwertuj dokumenty do tekstów
texts = [doc.page_content for doc in docs]

# Pobierz embeddingi
embeddings = client.predict(endpoint="databricks-gte-large-en", inputs={"inputs": texts})["embeddings"]

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-2832326452911439>, line 7
      4 client = get_deploy_client("databricks")
      6 # Konwertuj dokumenty do tekstów
----> 7 texts = [doc.page_content for doc in docs]
      9 # Pobierz embeddingi
     10 embeddings = client.predict(endpoint="databricks-gte-large-en", inputs={"inputs": texts})["embeddings"]

NameError: name 'docs' is not defined

In [0]:
import pandas as pd
from pyspark.sql import SparkSession



# Dane jako DataFrame
df = pd.DataFrame({
    "text": texts,
    "embedding": embeddings
})

# Konwersja do Spark DataFrame i zapis do UC

df.write.mode("overwrite").saveAsTable("sqlday_edbworkshop.default.vector_documents")

In [0]:
from mlflow.vectorsearch.client import VectorSearchClient

vsc = VectorSearchClient()

index = vsc.create_index(
    name="vector_index_documents",
    endpoint_name="databricks-endpoint-dbrx",
    embedding_dimension=len(embeddings[0]),
    source_table="sqlday_edbworkshop.default.vector_documents",
    input_column="text",
    embedding_column="embedding"
)

print("Indeks utworzony:", index.index_name)

In [0]:
query_result = index.query(
    query_text="Podaj najważniejsze informacje z dokumentu",
    columns=["text"],
    num_results=3
)

for i, r in enumerate(query_result['data']):
    print(f"{i+1}. {r['text'][:300]}\n")